In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../ethan/washington_outages.csv')

In [4]:
df

,Area Affected,index,Year,Month,Event Date and Time,Restoration Date and Time,Duration,Utility/Power Pool,NERC Region,Type of Disturbance,Loss (megawatts),Number of Customers Affected
0,King,2,2012,1.0,2012-01-19 07:00:00,2012-01-20 15:00:00,"32 Hours, 0 Minutes",Puget Sound Energy,WECC,Severe Weather - Winter Storm,1600,426000
1,Pierce,2,2012,1.0,2012-01-19 07:00:00,2012-01-20 15:00:00,"32 Hours, 0 Minutes",Puget Sound Energy,WECC,Severe Weather - Winter Storm,1600,426000
2,Thurston,2,2012,1.0,2012-01-19 07:00:00,2012-01-20 15:00:00,"32 Hours, 0 Minutes",Puget Sound Energy,WECC,Severe Weather - Winter Storm,1600,426000
3,WA,22,2013,5.0,2013-05-09 13:21:00,2013-05-09 16:21:00,"3 Hours, 0 Minutes",WECC,WECC,Electrical System Separation (Islanding),Unknown,Unknown
4,King,72,2013,11.0,2013-11-02 00:00:00,2013-11-04 06:00:00,"54 Hours, 0 Minutes",Puget Sound Energy,WECC,Severe Weather - Heavy Winds,Unknown,105000
...,...,...,...,...,...,...,...,...,...,...,...,...
68,Skagit,125,2018,12.0,2018-12-20 09:30:00,2018-12-20 17:00:00,"7 Hours, 30 Minutes",Puget Sound Energy,WECC,"Loss of electric service to more than 50,000 c...",Unknown,165000
69,Snohomish,125,2018,12.0,2018-12-20 09:30:00,2018-12-20 17:00:00,"7 Hours, 30 Minutes",Puget Sound Energy,WECC,"Loss of electric service to more than 50,000 c...",Unknown,165000
70,King,125,2018,12.0,2018-12-20 09:30:00,2018-12-20 17:00:00,"7 Hours, 30 Minutes",Puget Sound Energy,WECC,"Loss of electric service to more than 50,000 c...",Unknown,165000
71,Kitsap,125,2018,12.0,2018-12-20 09:30:00,2018-12-20 17:00:00,"7 Hours, 30 Minutes",Puget Sound Energy,WECC,"Loss of electric service to more than 50,000 c...",Unknown,165000


In [5]:
df.dtypes

Area Affected                    object
index                             int64
Year                              int64
Month                           float64
Event Date and Time              object
Restoration Date and Time        object
Duration                         object
Utility/Power Pool               object
NERC Region                      object
Type of Disturbance              object
Loss (megawatts)                 object
Number of Customers Affected     object
dtype: object

In [6]:
df_tweet = pd.read_csv('../ethan/washington_outage_tweets.csv')

In [14]:
df_tweet['outage'].describe()

count    248.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: outage, dtype: float64

In [12]:
df_tweet.dtypes

event_row     int64
timestamp    object
text         object
outage        int64
dtype: object